## Homework Grading
The goal of this is to develop a robust homework grading system which combines Google Colab + Jupyter Notebooks + GitHub Classroom + Otter Grader. 

For this example, we are going to download a zip file with a sample assignments.  However, if you end up grading from colab, you may consider directly mounting your google drive with the code below.  


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Build the Docker Container
I find it is best to customize your docker container and build it if grading locally. 

In [ ]:
#!cd ../docker && docker build . -t otter-helper:latest

### Setup for Grading

Make sure you have installed the dependencies in the `requirements.txt` file. 

`pip install -r requirements.txt`

Make sure you have all of the required files.

1.  Download assignment from github and put it in the `assignments/<course>/<assignment>`
2. Update the `/config/course/<course>.yml` file with the appropriate assignment file. 
3. Verify that the appropriate tests and data are in the associated `config/course/assignment/` folder. 
4. Verify that the roster is setup in `config/course/roster.xlsx`. 
5. Verify other configurations in  `/config/course/<course>.yml` are correct.  

If you want to delete the tmp grading folder set `delete_tmp` to `True`. 

### Set and Load the Configuration
We save configuration in a local file. 

In [2]:
#set to what you want to grade
course = 'sample-class' #use sample tests
assignment_id = 'github-classroom'  #use sample tests
#assignment_id = 'blackboard'  #use sample tests
#Set configuration
from pathlib import Path
import configparser, sys, os, importlib
import pandas as pd
 
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent #For running locally
#base_dir='/content/drive/My Drive/autograding'
modules_path = base_dir / 'modules' 
sys.path.append('../modules') # Not sure why appending the mudles path didn't work. 

#Load the autograding library
import autograde as ag
importlib.reload(ag)
cf = ag.get_config(course, assignment_id, base_dir)
cf

{'class_name': 'Sample Class',
 'class_id': 'sample-class',
 'message_complete': 'Your submission was successfully received and graded.<br>',
 'message_incomplete': 'If you get this message it is because you did not submit assignment. Please see the TA.<br>',
 'num_containers': 4,
 'ignore': ['.ipynb_checkpoints', '.DS_Store'],
 'requirements': 'config/sample-class/requirements.txt',
 'roster': 'config/sample-class/roster.xlsx',
 'assignments': {'blackboard': {'name': 'Sample assignment',
   'type': 'bb',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/blackboard/hidden-tests',
   'requirements': 'config/sample-class/requirements.txt',
   'seed': 42,
   'bb_column': 'blackboard_grade_test',
   'files': ['config/files/test_data_a.csv', 'config/files/test_data_b.csv']},
  'github-classroom': {'name': 'Sample assignment2',
   'type': 'gc',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/github-classroom/hidden-tests',
   'requirements': 'config/sample-class

### Prepare Grading
The will put either the Blackboard or the github classroom files in a format for use with Otter Grader.

cf = The config loaded earlier.
cleanup = True will delete the /tmp directory. 
submissions, if you just want to select specific entries. 
count = The number that you want to grade. 

TBD NEED TO MOVE THE FILES OVER. 

In [ ]:
files=ag.prepare_grade(cf,  cleanup=True)  
files

## Running the Grader 

This will launch a container, grade the assignment, and put the resulting in grades.csv. 

In [ ]:
!cd ../tmp && otter -j meta.json -t tests --image otter-helper:latest -v

## Prepare Grading For Upload
No you have graded all the students there is an output file in the /tmp directory with their grades.  The `prepare` command below will match the CSV file





In [9]:
#Aggregate JSON files for manual grading. 
blackboard=ag.prepare_blackboard_upload(cf, archive=True)
blackboard

complete: 6 
Incomplete: 0 
Total: 6


,Last Name,First Name,Username,Student ID,Last Access,Availability,blackboard_grade_test,Grading Notes,Notes Format,Feedback to Learner,Feedback Format
0,Name,Name,fails3,fails3,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML
1,Name,Name,passesAll,passesAll,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
2,Name,Name,fails2Hidden,fails2Hidden,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML
3,Name,Name,fails3Hidden,fails3Hidden,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
4,Name,Name,fails2,fails2,2020-08-01 22:08:00,Yes,4.0,NaN,NaN,Your submission was successfully received and ...,HTML
5,Name,Name,fails1,fails1,2020-08-01 22:08:00,Yes,4.0,NaN,NaN,Your submission was successfully received and ...,HTML


## Upload the Blackboard File 

And you are done. 